# 字典和集合

字典在 Python 中至关重要，即使没有直接使用，也会间接用到，因为 <mark>dict 类型是实现 Python 的基石</mark>

> 一些 Python 核心结构在内存中以字典的形式存在，比如类和实例属性、模块命名空间，以及函数的关键字

> 由于字典很重要，所以 Python 对字典做了高度优化，而且一直在改进，Python 字典高效的原因是使用了哈希表

> 除了字典之外，`set` 和 `frozenset` 也是基于哈希表，而且这两个类型的运算符和 API 相比于其他语言的集合更加丰富
>
> <span style="color:green"> Python 中的集合实现了集合论中的所有基本运算，包括并集、交集、子集测试等</span>

包括以下内容：
- 构建及处理 dict 和<mark>映射</mark>的现代语法, 包括增强的拆包和模式匹配
- 映射类型的常用方法
- 特别处理缺失键的情况
- 标准库中的 dict 变体
- set 和 frozenset 类型
- 哈希表对集合和字典行为的影响

> dict 和 set 的底层实现依赖于哈希表，不过 dict 的代码有两项重要的优化，节省内存，还能保留键的插入顺序

## 3.2 字典的现代句法

### 字典推导式

In [1]:
dial_codes = [
    (880, 'Bangladesh'),
    (55, 'Brazil'),
    (86, 'China'),
    (91, 'India'),
    (81, 'Japan'),
]

country_dial = {country: code for code, country in dial_codes}
print(country_dial)

{'Bangladesh': 880, 'Brazil': 55, 'China': 86, 'India': 91, 'Japan': 81}


In [2]:
{code: country.upper() 
 for country, code in sorted(country_dial.items())
 if code < 100
}

{55: 'BRAZIL', 86: 'CHINA', 91: 'INDIA', 81: 'JAPAN'}

### 映射拆包

Python3.5 开始对映射拆包功能做了增强

- 调用函数时，不止一个参数可以使用 `**`, 但是<mark>所有的键都要是字符串，而且在所有的参数重是唯一的</mark>
- `**` 可以在 dict 字面量中使用，同样可以多次使用

In [3]:
def dump(**kwargs):
    return kwargs

dump(**{'x': 1}, y=2, **{'z': 3, 't': 4})

{'x': 1, 'y': 2, 'z': 3, 't': 4}

In [1]:
{'a': 0, **{'x': 1}, 'y': 2, **{'z': 3, 'x': 4}}

{'a': 0, 'x': 4, 'y': 2, 'z': 3}

### 使用 | 合并映射

In [4]:
d1 = dict(a=1, b=2)
d2 = dict(c=3, d=4, a=6)

print(d1 | d2)

{'a': 6, 'b': 2, 'c': 3, 'd': 4}


更新后的 a 是 6

通常来说，新映射的类型与左操作数的类型相同

可以通过就地修改操作，使用 `|=` 更改当前现有的映射

## 3.3 使用模式匹配处理映射

`match/case` 语句可以匹配映射。

不同类型的模式可以组合和嵌套，模式匹配是一种强大的工具，借助析构可以处理嵌套的映射和序列等结构化记录

> 经常需要从 JSON API 和具有半结构化模式的数据库（MongoDB、PG）读取数据

In [6]:
# 从出版物记录中提取创作者的名字

def get_creators(record: dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:
            return names
        case {'type': 'book', 'api': 1, 'author': name}:
            return [name]
        case {'type': 'book'}:
            raise ValueError(f"Invalid 'book' record: {record!r}")
        case {'type': 'movie', 'director': name}:
            return [name]
        case _:
            raise ValueError(f"Invalid record: {record!r}")

下面做一个小小的测试，测试一下 get_creators 函数

In [10]:
b1 = dict(api=1, author='Douglas Adams', type='book', title='war of the worlds')

author = get_creators(b1)
print(author)

from collections import OrderedDict
b2 = OrderedDict(api=2, type='book', title='war of the worlds', authors='Douglas Adams'.split())
authors = get_creators(b2)
print(authors)

get_creators({'type': 'book', 'pages': 100})

['Douglas Adams']
['Douglas', 'Adams']


ValueError: Invalid 'book' record: {'type': 'book', 'pages': 100}

<mark>模式中的键的顺序无关紧要</mark>

<span style="color: green"> 与序列模式不同，就算只有部分匹配，映射模式也算匹配成功 </span>


<span style="color:green"> 没有必要使用 `**extra` 匹配多出的键值对，若想把多余的键值对捕获到一个 dict 中，可以在一个变量前面加上 `**`, 不过需要放在最后面 </span> 

In [11]:
food = dict(category='fruit', name='apple', price=1.99)
match food:
    case {'category': 'fruit', **details}:
        print(f'fruit details: {details}')

fruit details: {'name': 'apple', 'price': 1.99}


> 模式匹配不自动处理缺失的键，因为模式匹配始终使用 `d.get(key, sentinel)` 方法，其中，sentinel 是特殊的标记，不会出现在用户数据中。

## 3.4 映射类型的标准 API

<span style="color:red"> `isinstance` 经常用来检查是否满足抽象基类的要求 </span>

自定义映射类型的键必须是可哈希的

### 「可哈希」指什么

> 如果一个对象的 **哈希码** 在整个生命周期内 **永不改变**（依靠 `__hash__()` 方法），而且**可与其他对象比较**（依靠 `__eq__()` 方法），那么这个对象就是可哈希的。

可哈希的对象：数值、字符串、字符、不可变的容器

当键 k 不存在时，`d[k]` 抛出错误，当把 `d[k]` 换成 `d.get(k, default)`, 若不存在 `d[k]`，则返回 default。

也可以使用 `d.setdefault(k, v)`

## 3.5 自动处理缺失的键

<mark>TODO</mark>

## 3.6 dict 的变体

### `collections.OrderedDict`

Python3.6 起, 内置的 dict 也保留了键的顺序。

dict 和 OrderedDict 的差异
- OrderedDict 的等值检查考虑顺序
- OrderedDict 多了一个 `move_to_end()` 方法，便于把元素的位置移到某一端
- OrderedDict 目的是方便执行重新排序操作，空间利用率、迭代速度和更新操作的性能是次要的
- dict 用于执行映射操作，插入顺序是次要的
- OrderedDict 处理频繁重新排序操作的效果比 dict 好，因此适用于跟踪近期存取情况（**例如 LRU 缓存**）

### `collections.ChainMap`

ChainMap 存放一组映射，可以作为一个整体来搜索。查找按照输入映射在构造函数调用中出现的顺序执行，一旦在某个映射中找到指定的键，就结束

In [1]:
d1 = dict(a=1, b=3)
d2 = dict(a=2, b=4, c=6)

from collections import ChainMap
chain = ChainMap(d1, d2)

print(chain['a'])
print(chain['c'])

1
6


- ChainMap 实例不复制输入映射，而是存放映射的引用。

- ChainMap 的更新或插入操作只影响第一个输入映射。

In [3]:
chain['c'] = -1

print(d1)
print(d2)

{'a': 1, 'b': 3, 'c': -1}
{'a': 2, 'b': 4, 'c': 6}


### `collections.Counter`

它可以对键计数进行映射。更新现有的 key，计数随之增加。

<mark>可以统计可哈希对象的实例数量</mark>

`Counter` 实现了组合计数的 `+` 和 `-` 运算符，以及其他一些有用的方法，比如`most_common([n])`

In [1]:
from collections import Counter

ct = Counter('abracadabra')
print(ct)

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})


In [2]:
ct.update('aaaaaaazzz')
print(ct)

Counter({'a': 12, 'z': 3, 'b': 2, 'r': 2, 'c': 1, 'd': 1})


In [4]:
ct.most_common(4)

[('a', 12), ('z', 3), ('b', 2), ('r', 2)]

### shelve.Shelf

shelve 模块持久存储<mark>字符串键</mark>

- `shelve.open` 返回 `shelve.Shelf` 实例，这是一个简单的键值 DBM 数据库
- `shelve.Shelf` 提供了一些 `I/O` 管理方法，例如 sync 和 close
- `Shelf` 实例是上下文管理器，可以使用 with 块确保在使用后关闭
- 键必须是字符串

<span style="color: green"> 创建新的映射类型，最好扩展 `collections.UserDict`，而不是 `dict`.</span>

- 继承 `UserDict` 主要原因是：内置的 dict 在实现上走了一些捷径，如果继承 dict, 那就不得不覆盖一些方法。

## 3.7 不可变映射

> 标准库提供的映射类型都是可变的，但有时候需要防止用户更改映射

types 模块提供了 `MappingProxyType` 是一个包装类，把传入的映射包装成一个 `mappingproxy` 实例，只可以读取（类似只读类型）

也就是说，对原映射的更新体现在 `mapppingproxy` 实例上，但是不能通过它更改映射

In [5]:
from types import MappingProxyType
d = {'a': 1, 'b': 2}
d_proxy = MappingProxyType(d)
print(d_proxy)

print(d_proxy['a'])

{'a': 1, 'b': 2}
1


In [6]:
d_proxy['b'] = 3

TypeError: 'mappingproxy' object does not support item assignment

In [7]:
d['b'] = 3

print(d_proxy)
print(d['b'])

{'a': 1, 'b': 3}
3


## 3.8 字典视图

> 通过视图可以对字典执行一些高性能操作，免去复制数据的麻烦

- dict的实例方法 `.keys()`, `.values()` 和 `.items()` 分别返回 dict_keys, dict_values 和 dict_items 类的实例。

- 这些字典视图是 dict 内部实现使用的数据结构的只读投影

In [8]:
d = dict(a=10, b=20, c=30)
values = d.values()

print(values)
print(len(values))

dict_values([10, 20, 30])


<span style="color:green"> 视图是可迭代对象，方便构建列表，以及实现了 `__reversed__` 方法 </span>

In [9]:
print(list(values))

print(reversed(values))

[10, 20, 30]


<mark> 不能使用 `[]` 获取视图中的项 </mark>

视图对象是动态代理，更新原 dict 对象后，现有视图立即就能看到变化

`dict_keys`, `dict_values`, `dict_items` 是内部类，不同通过任何模块获取到，尽管可以获取实例，但是无法在 Python 代码中自己动手创建

In [10]:
values_class = type({}.values())

v = values_class()

TypeError: cannot create 'dict_values' instances

- `dict_values` 类是最简单的字典视图，只实现了 `__len__`, `__iter__` 和 `__reversed__` 三个特殊方法。
- `dict_keys` 和 `dict_items` 还实现了多个集合方法，类似于 `frozenset`。

## 3.9 dict 的实现方式对实践的影响

> Python 使用哈希表实现 dict，字典效率非常高，但对实践也有一定的影响

- 键必须是可哈希的对象（需要正确实现 `__hash__` 和 `__eq__` 方法
- 通过键访问项的速度非常快。一个包含数百万个键的 dict 对象通过哈希码就可以直接定位，然后找出索引在哈希表中的偏移量，稍微几次就能匹配到，所以开销并不大
- Cpython3.6 中，dict 的内存布局更为紧凑，键的顺序也得以保留
- 尽管采用了新的紧凑布局，但是字典仍然占用了大量的内存，不可避免。<mark> 最紧凑的内部数据结构是指向项的指针的数组</mark>, 而哈希表中的条目存储的数据更多，为了保证效率，至少需要把哈希表中的三分之一的行留空。
- 为了节省内存，不要在 `__init__` 方法之外创建实例属性。（Python 默认在特殊的 __dict__ 属性中存储实例属性，这个属性的值是一个字典依附在各个实例上

## 3.10 集合论 

set 和 frozenset 是 Python3 的内置集合类型

集合是一组唯一的对象，可以用来进行去除重复项

In [11]:
l = ['spam', 'spam', 'eggs', 'bacon', 'eggs']

print(set(l))

print(list(set(l)))

{'bacon', 'eggs', 'spam'}
['bacon', 'eggs', 'spam']


> 如果想去除重复项，同时保留每一项首次出现位置的顺序，那么使用普通的 dict 即可

In [12]:
k = dict.fromkeys(l).keys()
print(k)

print(list(k))

dict_keys(['spam', 'eggs', 'bacon'])
['spam', 'eggs', 'bacon']


集合元素必须是可哈希的对象。

- set 类型不可哈希
- frozenset 类型可哈希

集合类型实现了许多集合运算

- a & b 计算交集
- a | b 计算并集
- a - b 计算差集
- a ^ b 计算对称差集

<span style="color: green"> 巧妙使用集合运算可以减少代码行数，也能缩短 Python 程序的运行时间，同时还能少写一些循环和条件判断 </span>

比如现在一个集合中存储了大量的电子邮件 haystack, 还有一个集合存储少量电子邮件 needles，想统计 needles 中多少邮件出现在 haystack 中

- **方式一**

In [ ]:
needles = haystack = {}
found = len(needles & haystack)

- **方式二**

In [ ]:
found = 0
for n in needles:
    if n in haystack:
        found += 1

上面两种方式都可以实现统计，但是方法一的运行速度比方法二稍快

如果上面的 needles 和 haystack 是任何可以迭代的对象，而不是集合，也可以快速构建

```python
found = len(set(needles) & set(haystack))

# 或者另一种方式
found = len(set(needles).intersection(haystack))
```

### set 字面量

set 字面量的句法和集合的数学表示法几乎一样，如 {1}, {1, 2}, 空的set 必须写成`set()`

> 如果空的 set 写成 {}, 则创建的是 dict
>
> 与调用构造函数相比（`set([1,2,3])`），<mark>使用 set 字面量句法不仅速度快, 而且更具可读性</mark>

## 3.11 集合的实现方式对实践的影响

set 和 frozenset 都使用哈希表实现

- 集合元素必须是可哈希的对象
- 元素的顺序取决于插入顺序，但是顺序对于集合没什么意义，也得不到保障

`dict_keys` 和 `dict_items` 实现了一些特殊的方法，支持强大的集合运算：包括 & 交集、｜ 并集、- 差集 和 ^ 对称差集

In [14]:
# 使用 & 运算符轻易获取两个字典都有的key
d1 = dict(a=1, b=2, c=3, d=4)
d2 = dict(b=20, d=40, e=50)

print(d1.keys() & d2.keys())

{'d', 'b'}


可以看到，使用 `&` 运算符返回一个 set，而且字典视图的集合运算兼容 set 实例

In [15]:
s = {'a', 'e', 'i'}

print(d1.keys() & s)
print(d1.keys() | s)

{'a'}
{'c', 'e', 'a', 'i', 'd', 'b'}


> 仅当 dict 中的所有值均可哈希时，`dict_items` 视图才能当作集合使用
>
> 使用集合运算处理视图可以省去大量循环和条件判断

## 3.13 小结

- 字典是 Python 的基石，目前支持 **拆包，模式匹配和字典推导式
- collections 模块提供了一些专门的映射 `defaultdict`, `ChainMap` 和 `Counter`

- 多数映射具有两个强大的方法：`setdefault` 和 `update`
  - `setdefault` 可以更新 存放可变值的项（如 list），从而避免再次搜索相同的 key
  - `update` 可以批量插入或者覆盖项，项可以来自提供键值对的可迭代对象，也可以来自关键字参数
- `__missing__` 方法是一个巧妙的狗子，利用它可以自定义 d[k] 句法


---
> PyCon 2017 "The Dictionary Even Mightier" 是 "The Mighty Dictionarry" 的续集，用动画演示哈希碰撞，同时作者的 "Modern Dictionaries" 对 dict 的内部机制剖析更为深入


---
小技巧

> JSON几乎是Python句法的子集，除了 true, false, null 等值的拼写不一样外，几乎是兼容的
>
> 所以可以在 Python 的全局空间中为 Python 的 True, False 和 None 添加兼容 JSON 的别名，这样就可以直接把 JSON 粘到 Python 的控制台了

In [16]:
true, false, null = True, False, None

friut = {
    "type": "banana",
    "avg_weight": 123.2,
    "ediable_peel": false,
    "issues": null,
}

print(friut)

{'type': 'banana', 'avg_weight': 123.2, 'ediable_peel': False, 'issues': None}
